Creating a solar system simulator for my fantasy universe.

In [136]:
# Base units are in kg, m, and s
import numpy as np
import math
import pygame
import sys

def calc_escape_velocity(mass, distance):
    return np.sqrt((2 * G * mass)/distance)

def calc_esc_radius(mass, V_esc):
    # Should give swarzchild radius when V_esc = c
    return (2 * G * mass)/V_esc **2

def calc_radial_velocity(radius, period):
    return (2 * np.pi * radius)/period

def calc_radius_from_vel(rad_vel, period):
    return (rad_vel * period)/ (2 * np.pi)
   
def calc_period(distance, mass):
    return 2 * np.pi *(math.sqrt(distance**3/(G * mass)))

def calc_grav_time_dilation(s_radius, distance):
    return (1 - (s_radius / distance))**0.5
    
def calc_vel_time_dilation(velocity):
    return 1/(math.sqrt(1-(velocity/c)**2))
        
def rel_velocity_addition(v1, v2):
    return (v1 + v2 ) / (1 + (v1 * v2)/(c**2))

In [138]:
# Real life units for relations
G          = 6.674e-11      # N * m^2 * kg^-2
c          = 2.99e+8        # m * s^-1
sol_mass   = 1.99e+30       # kg
earth_mass = 5.97e+24       # kg
earth_r    = 6.34e6         # m
au         = 1.49e+11       # m
year       = 3600*24*365.25 # s
second     = 1
month      = 3600*24*28
lightyear  = c * year       # m

#Fictional objects
# Red Dwarf stars are between 0.08 to 0.6 times the mass of the sun, 
anta_mass      = 0.2 * sol_mass
anta_r         = 0.3 * 6.96e8

marab_r        = 2.02e7
marab_d        = 0.85 * au 
marab_mass     = 10 * earth_mass
marab_year     = calc_period(marab_d, anta_mass)
marab_yvel     = calc_radial_velocity(marab_d, marab_year)

sada_mass      = 7.348e+22
sada_d         = 4.84e+8
sada_r         = 1731
sada_month     = calc_period(sada_d, marab_mass)
sada_yvel      = marab_yvel + calc_radial_velocity(sada_d, sada_month)

bh_mass        = 100 * sol_mass
s_radius       = calc_esc_radius(bh_mass, c)

print(f"Marab's year is equal to {marab_year/year:.2f} earth years, or {marab_year/(3600*24):.2f} days.") 
print(f"And it's month is equal to {sada_month/month:.2f} months or {sada_month/(3600*24):.2f} days.")
print(f"Meaning there are {marab_year/sada_month:.2f} months per year.")
print(f"Marab is moving at a speed of {marab_yvel:.2f}m/s around Anta.")

#print(f"Mercury's radial velocity is: {calc_radial_velocity(0.39*au, 0.24*year):.2f}m/s")

Marab's year is equal to 1.74 earth years, or 635.97 days.
And it's month is equal to 0.44 months or 12.27 days.
Meaning there are 51.84 months per year.
Marab is moving at a speed of 14482.12m/s around Anta.


In [151]:
pygame.init()

time_scales = [
    (1e9 * year, "billion year"),
    (1e6 * year, "million year"),
    (1e3 * year, "thousand year"),
    (year, "year"),
    (1, "second"),
    (1e-6, "microsecond"),
    (1e-9, "nanosecond"),
    (1e-12, "picosecond"),
    (5.39e-44, "Planck second")  
]

# Creating a figure and axis
width, height = 1280, 700
win = pygame.display.set_mode((width, height))
pygame.display.set_caption("Orbit Simulation")

# Color library
white        = (255,255,255)
black        = (0, 0, 0)
dark_gray    = (89, 89, 89)
yellow       = (255, 255, 0)
blue         = (0, 0, 255)
red          = (255, 0, 0)
cosmic_latte = (255, 248, 231)
gray         = (222, 222, 222)
light_blue   = (204,212,255)
brown        = (77,38,0)

# Font
font         = pygame.font.SysFont("ariel", 16)
pause_bttn   = pygame.Rect(20, 20, 100, 50)

# Size and space scaling
year_ticks    = marab_year/10000
month_ticks   = sada_month/900

year_zoom    = 340 / marab_d
month_zoom   = 340 / sada_d

dt           = month_ticks
scale        = month_zoom

object_texts = {}

class Planet: 
    def __init__(self, x, y, radius, color, mass):
        self.x               = x
        self.y               = y
        self.radius          = radius # in pixels
        self.color           = color
        self.mass            = mass
        self.rel_clock       = 0
        self.intl_clock      = 0
        
        # Should also make light object functionality
        self.sun             = False
        self.distance_to_sun = 1 # Avoiding division by 0
        
        # To draw event horizons and handle collisions
        self.bh              = False
        self.absorbed        = False
        
        self.orbit           = []
        
        self.x_vel           = 0 # m/s
        self.y_vel           = 0 # m/s
        self.speed_magnitude = math.sqrt(self.x_vel**2 + self.y_vel**2)
                
            
    def draw_event_horizon(self, planets):
        self.event_horizon = calc_esc_radius(self.mass, c)
        center             = (scale * self.x + (width/2), scale * self.y + (height/2))
        for planet in planets:
            if self.bh:  
                # Draw event horizon
                pygame.draw.circle(win, black, center, int(scale*self.event_horizon), 2)
                
                # Draw photon sphere
                pygame.draw.circle(win, yellow, center, int(scale * 1.5 * self.event_horizon),2)
                 

    
    def attraction(self, other):
        other_x, other_y = other.x, other.y
        distance_x       = other_x - self.x
        distance_y       = other_y - self.y
        distance         = math.sqrt(distance_x ** 2 + distance_y ** 2)
        
        if other.sun:
            self.distance_to_sun = distance
        elif other.bh:
            self.distance_to_sun = distance
        
        mu      = self.mass * other.mass
        force   = G * mu / distance ** 2
        theta   = math.atan2(distance_y, distance_x)
        force_x = math.cos(theta) * force
        force_y = math.sin(theta) * force
        return force_x, force_y
    
    
    def update_position(self, planets):
        # Initial forces at 0
        total_fx = total_fy = 0
        
        for planet in planets:
            # Avoiding division by 0
            if self == planet:
                continue
            fx, fy    = self.attraction(planet)
            total_fx += fx
            total_fy += fy
            
        # Calculate acceleration
        accel_x = total_fx / self.mass
        accel_y = total_fy / self.mass
    
        # Update velocity components
        self.x_vel += accel_x * dt
        self.y_vel += accel_y * dt
        
        self.x += self.x_vel * dt
        self.y += self.y_vel * dt
        
        self.orbit.append((self.x, self.y))
        
        self.speed_magnitude = math.sqrt(self.x_vel**2 + self.y_vel**2)
        if self.speed_magnitude >= c:
            # Cap velocity
            self.x_vel = self.x_vel / self.speed_magnitude * (c * 0.9999)
            self.y_vel = self.y_vel / self.speed_magnitude * (c * 0.9999)
    
    # Have things fall into the black hole
    def collide(self, other, planets):
        other.event_horizon    = calc_esc_radius(other.mass, c)
        distance_x, distance_y = self.x - other.x, self.y - other.y
        distance               = math.sqrt(distance_x**2 + distance_y**2)
        
        if distance <= other.event_horizon:
            other.mass += self.mass
            planets.remove(self)
                        
                
    def draw(self, planets, win, is_1d=False):
        if is_1d:
            center_x = width/2
            
            for planet in planets:
                x = center_x + planet.x * scale
                pygame.draw.circle(win, planet.color, (int(x), height // 2), planet.radius)
            
        else:
            x = self.x * scale + width/2
            y = self.y * scale + height/2


            if len(self.orbit) > 2:
                # Keep only the most recent points
                if len(self.orbit) > 99999:
                    self.orbit = self.orbit[-99999:]

                updated_points = []

                for point in self.orbit:
                    x, y = point
                    x = x * scale + width/2
                    y = y * scale + height/2
                    updated_points.append((x,y))

                pygame.draw.lines(win, self.color, False, updated_points, 1)

            pygame.draw.circle(win, self.color, (x,y), self.radius)
            
            
def main():
    run            = True
    paused         = False
    clock          = pygame.time.Clock()
    time_counter   = 0
    relative_time  = 0
    time_diff      = 0
    
    # Planet(x, y, radius, color, mass)
    
    #anta         = Planet(0, 0, int(scale * anta_r), yellow, anta_mass)
    anta         = Planet(0, 0, 7, yellow, anta_mass)
    anta.sun     = True
    
    
    #marab        = Planet(marab_d, 0, int(scale * marab_r), light_blue, marab_mass)
    marab        = Planet(marab_d, 0, 3, light_blue, marab_mass)
    marab.y_vel  = -marab_yvel
    
    #sada         = Planet(marab_d + sada_d, 0, int(scale * sada_r), white, sada_mass)
    sada         = Planet(marab_d + sada_d, 0, 1, white, sada_mass)
    sada.y_vel   = -sada_yvel
    
    marab2       = Planet(0, 0, 7, light_blue, marab_mass)
    
    sada2        = Planet(sada_d, 0, 3, white, sada_mass)
    sada2.y_vel  = -(sada_yvel - marab_yvel)
    
    # testing a second moon
    moon2        = Planet(sada_d/4, 0, 3, white, sada_mass * 2)
    moon2.y_vel  = - calc_radial_velocity(sada_d/4, calc_period(sada_d/4, marab_mass))
    
    moon3        = Planet(sada_d/2, 0, 3, white, sada_mass/3)
    moon3.y_vel  = - calc_radial_velocity(sada_d/2, calc_period(sada_d/2, marab_mass))
    
    # Resonant system: Anya and Marab orbit a central black hole
    # calc_radial_velocity(distance, period)
    # calc_period(distance, mass)
    
    t_bh         = Planet(0, 0, int(scale * bh_mass), black, bh_mass)
    t_bh.sun     = True
    t_bh.bh      = True
    
    anta2_d      = 0.5 * marab_d
    anta2_yvel   = calc_radial_velocity(anta2_d, calc_period(anta2_d, bh_mass))
    
    anta2        = Planet(anta2_d, 0, int(scale * anta_r), yellow, anta_mass)
    anta2.y_vel  = -anta2_yvel
    
    marab3       = Planet(anta2_d + marab_d, 0, int(scale * marab_r), light_blue, marab_mass)
    marab3.y_vel = -(marab_yvel + anta2_yvel) 
    
    # Test systems
    story_sys    = [anta, marab, sada]
    month_sys    = [marab2, sada2, moon2, moon3]
    bh_sys       = [t_bh, anta2, marab3]
    
    planets      = month_sys
    
    while run:
        clock.tick(30)
        win.fill(dark_gray)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    if pause_bttn.collidepoint(event.pos):
                        paused = not paused
        if not paused:
            # Change the time scale as the counters gets higher
            for exponent, unit in time_scales:
                # Check if time_counter or relative_time is greater than the current scale
                if time_counter >= exponent:
                    # Use the current scale for rendering
                    if unit == "Planck second":
                        time_text     = font.render(f"Inertial Clock in {unit}s: {time_counter/exponent:.3e}", 1, white)
                    else:
                        time_text = font.render(f"Inertial Clock in {unit}s: {time_counter/exponent:.3f}", 1, white)
                    break

            for planet in planets:
                planet.update_position(planets)
                planet.draw(planets, win, is_1d=False)
                planet.draw_event_horizon(planets)
                for other in planets:
                    if planet != other:
                        planet.collide(other, planets)
                        
            
        pygame.display.update()
                
    pygame.quit
    
main()

SystemExit: 